In [27]:
import pandas as pd
import numpy as np
import os

def preprocess_file(path):
    file_extension = os.path.splitext(path)[1].lower()

    if file_extension == ".xlsx":
        # Load Excel file
        df = pd.read_excel(path)
    elif file_extension == ".csv":
        # Load CSV file
        df = pd.read_csv(path)
    else:
        raise ValueError("Unsupported file format. Please provide a .xlsx or .csv file.")

    # View head
#     print(df.head())

    # --- Check missing values ---
    print("\nMissing values per column:\n", df.isnull().sum())

    # --- Impute numeric with mean ---
    numeric_cols = df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].mean())

    # --- Impute categorical with mode ---
    categorical_cols = df.select_dtypes(include='object').columns
    for col in categorical_cols:
        mode_val = df[col].mode()
        if not mode_val.empty:
            df[col] = df[col].fillna(mode_val[0])
    for col in categorical_cols:
        df[col] = df[col].astype(str).str.strip()

    # --- Clean column names ---
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(r"[^\w\s]", "", regex=True)
        .str.replace(r"\s+", "_", regex=True)
    )

    # Optional: Check again after cleaning
#     print("\nNA count after cleaning:\n", df.isnull().sum())
#     print("\nCleaned column names:\n", df.columns.tolist())
    return df


In [31]:
hdi = preprocess_file('HDI.csv')
wb = preprocess_file('WorldBank.xlsx')

columns_to_keep = [
    'iso3', 'country', 'hdicode', 'region','hdi_rank_2021','hdi_2019','hdi_2020',
    'hdi_2021', 'le_2019','le_2020', 'le_2021', 'eys_2019','eys_2020','eys_2021', 'mys_2019','mys_2020','mys_2021', 'gnipc_2019',
    'gnipc_2020','gnipc_2021','gii_rank_2021', 'gii_2019','gii_2020','gii_2021', 
    'co2_prod_2019','co2_prod_2020','co2_prod_2021', 'mf_2020','mf_2021'
]

hdi_small = hdi[columns_to_keep]


Missing values per column:
 iso3              0
country           0
hdicode          15
region           55
hdi_rank_2021    15
                 ..
mf_2017          38
mf_2018          38
mf_2019          38
mf_2020          38
mf_2021          38
Length: 1008, dtype: int64

Missing values per column:
 Country Name                                                       0
Country Code                                                       0
Region                                                             0
IncomeGroup                                                        0
Year                                                               0
Birth rate, crude (per 1,000 people)                            1009
Death rate, crude (per 1,000 people)                            1033
Electric power consumption (kWh per capita)                     6601
GDP (USD)                                                       2871
GDP per capita (USD)                                            2874
Indiv

In [32]:
import pandas as pd
from sqlalchemy import create_engine

# --- Configuration ---
# Your local SQL Server settings
server = 'DESKTOP-L5SAUMR\SQLEXPRESS'  # Change if needed (e.g., 'localhost\\SQLEXPRESS')
database = 'master'
driver = 'ODBC Driver 17 for SQL Server'  # Check Azure Data Studio or ODBC panel if unsure

# Use Windows Authentication (trusted connection)
connection_string = f"mssql+pyodbc://{server}/{database}?trusted_connection=yes&driver={driver}"
engine = create_engine(connection_string)

print("Loading datasets...")
worldbank_df = wb
hdi_df = hdi_small

# Optional: Preview
# print("WorldBank preview:\n", worldbank_df.head())
print("HDI preview:\n", hdi_df.head())

# --- Upload to SQL Server ---
print("Uploading to SQL Server...")
# worldbank_df.to_sql('WorldBank', con=engine, if_exists='replace', index=False)
hdi_small.to_sql('HDI', con=engine, if_exists='replace', index=False)

print("✅ Upload complete! Check Azure Data Studio to verify.")


Loading datasets...
HDI preview:
   iso3               country    hdicode region  hdi_rank_2021  hdi_2019  \
0  AFG           Afghanistan        Low     SA          180.0     0.488   
1  AGO                Angola     Medium    SSA          148.0     0.595   
2  ALB               Albania       High    ECA           67.0     0.810   
3  AND               Andorra  Very High    SSA           40.0     0.873   
4  ARE  United Arab Emirates  Very High     AS           26.0     0.920   

   hdi_2020  hdi_2021  le_2019  le_2020  ...    gnipc_2021  gii_rank_2021  \
0     0.483     0.478  63.5645  62.5751  ...   1824.190915     167.000000   
1     0.590     0.586  62.4484  62.2612  ...   5465.617791     136.000000   
2     0.794     0.796  79.2825  76.9893  ...  14131.110390      39.000000   
3     0.848     0.858  83.0039  79.0234  ...  51166.626610      85.376471   
4     0.912     0.911  79.7262  78.9457  ...  62573.591810      11.000000   

   gii_2019  gii_2020  gii_2021  co2_prod_2019  co2_